In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from collections import defaultdict
from nltk.tokenize import word_tokenize 
from collections import Counter
import re


In [2]:
movies = pd.read_csv('Data/cleaned_overview_df.csv')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9863 entries, 0 to 9862
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   original_title     9863 non-null   object 
 1   id                 9863 non-null   int64  
 2   original_language  9863 non-null   object 
 3   popularity         9863 non-null   float64
 4   release_date       9839 non-null   object 
 5   vote_average       9863 non-null   float64
 6   vote_count         9863 non-null   int64  
 7   genre              9863 non-null   object 
 8   overview           9863 non-null   object 
 9   revenue            9863 non-null   int64  
 10  runtime            9858 non-null   float64
 11  tagline            7077 non-null   object 
 12  summary_length     9863 non-null   int64  
 13  Unnamed: 15        9863 non-null   int64  
 14  Action             9863 non-null   int64  
 15  Adventure          9863 non-null   int64  
 16  Animation          9863 

In [3]:
too_general = ['Animation', 'TV Movie', 'Adventure', 'War', 'Family', 'Thriller'] 
movie_genre = [i.strip('[]').split(",") for i in movies.genre]
for movie in (movie_genre):
    for number, genres in enumerate(movie):
        movie[number] = genres.strip("' '")
movies['genre'] =  movie_genre

In [4]:
new = []
for i, genres in enumerate(movies['genre']):
    for genre in genres:
        if genre in too_general:
            genres.remove(genre)
    movies['genre'][i] = genres

C:\Users\sangr\AppData\Local\Temp/ipykernel_17076/2544758649.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['genre'][i] = genres


In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from skmultilearn.problem_transform import BinaryRelevance
brelevance = BinaryRelevance(classifier=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), require_dense=[True, True])


In [6]:
movies_df = movies[~movies['genre'].isin([[''], []])]

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X= movies_df['overview']
y = movies_df['Action']
y = y.values.reshape(-1,1)
X_train_count = vectorizer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_train_count,y, test_size = .3, random_state = 42)

In [18]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
binary_rel_clf = BinaryRelevance(MultinomialNB())
binary_rel_clf.fit(X_train, y_train)
br_prediction = binary_rel_clf.predict(X_test)
br_prediction.toarray()
accuracy_score(br_prediction.toarray(), y_test)

0.8090783090783091

In [17]:
accuracy_score(br_prediction.toarray(), y_test)


0.8090783090783091

In [ ]:
br_prediction

In [ ]:
movies_df